In [1]:
import pandas as pd

In [3]:
data = pd.read_csv("/data1-8tb-blk/sp2023stock/Insight-Wave/StockPrediction/data/Fundamental+Technical Data/STOCK_DATA_WEEKLY/ADVANC.csv")
data = data['Close']

In [5]:
data[0]

196.5